In [1]:
import os
import re

import urllib.request
import tiktoken
from importlib.metadata import version

In [3]:
with open("../data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])


Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [4]:
text= "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result1 = [item.strip() for item in result if item.strip()]
result2 = [item for item in result if item.strip()]
print(result1)
print(result2)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']
['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [5]:
preprocess = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocess = [item for item in preprocess if item.strip()]
print(len(preprocess))
all_words =  sorted(set(preprocess))
vocab_size= len(all_words)
print(vocab_size)

4690
1130


In [6]:
vocabs={vocab: idx for idx, vocab in enumerate(all_words)}

## Simple tokenizer

In [7]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.word2idx = vocab
        self.idx2word = {idx: word for word, idx in vocab.items()}
        self.size= len(vocab)+1
            
    def encode(self, text):
        words = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        words = [item for item in words if item.strip()]
        return [self.word2idx[word] for word in words] 
        
    def decode(self, ids):
        words=[self.idx2word[idx] for idx in ids]
        sentence=' '.join(words)
        sentence = re.sub(r'\s([,.?!"()\'])', r'\1', sentence)
        return sentence
        
    def __len__(self):
        return self.size
    
        

In [8]:
tokenizer  = SimpleTokenizerV1(vocabs)
text = """"It's the last he painted, you know,"
        Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))


[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [9]:
text = " Hello, do you like the tea"
print(tokenizer.encode(text))

KeyError: 'Hello'

In [10]:
all_words = sorted(list(set(preprocess)))
all_words.extend(["<|endoftext|>","<|unk|>"])
vocab = {word: idx for idx, word in enumerate(all_words)}
print(len(vocab))

1132


In [11]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(i, item)

0 ('younger', 1127)
1 ('your', 1128)
2 ('yourself', 1129)
3 ('<|endoftext|>', 1130)
4 ('<|unk|>', 1131)


In [12]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.word2idx = vocab
        self.idx2word = {idx: word for word, idx in vocab.items()}
        self.size= len(vocab)
            
    def encode(self, text):
        words = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        words = [item for item in words if item.strip()]
        return [self.word2idx.get(word, self.size-1) for word in words]
        
    def decode(self, ids):
        words=[self.idx2word[idx] for idx in ids]
        sentence=' '.join(words)
        sentence = re.sub(r'\s([,.?!"()\'])', r'\1', sentence)
        return sentence
        
    def __len__(self):
        return self.size

In [13]:
text1="Hello, do you like tea"
text2="In the sunlut terraces of the palace."
text = " <|endoftext|> ".join((text1,text2))
print(text)

Hello, do you like tea <|endoftext|> In the sunlut terraces of the palace.


In [14]:
tokenizer2=SimpleTokenizerV2(vocab)
ids_2=tokenizer2.encode(text)
print(ids_2)
print(tokenizer2.decode(ids_2))

[1131, 5, 355, 1126, 628, 975, 1130, 55, 988, 1131, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea <|endoftext|> In the <|unk|> terraces of the <|unk|>.


## Byte pair encoding

In [15]:
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.9.0


In [16]:
tokenizer = tiktoken.get_encoding('gpt2')

In [17]:
text = (
    "Hello, do you like tea <|endoftext|> In the sunlut terraces"
    "of someunknownPlace."
    )
idxs = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(idxs)

[15496, 11, 466, 345, 588, 8887, 220, 50256, 554, 262, 4252, 75, 315, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [18]:
strings=tokenizer.decode(idxs)
print(strings)

Hello, do you like tea <|endoftext|> In the sunlut terracesof someunknownPlace.


In [19]:
text="AKwirw ier"
ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(ids)
decode_msg=tokenizer.decode(ids)
print(decode_msg)

[10206, 86, 343, 86, 220, 959]
AKwirw ier


In [25]:
with open("../data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text= f.read()
enc_text=tokenizer.encode(raw_text)
print(len(enc_text))


5145


In [26]:
enc_sample=enc_text[50:]

In [27]:
context_size=4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f'x:  {x}')
print(f'y:       {y}')

x:  [290, 4920, 2241, 287]
y:       [4920, 2241, 287, 257]


In [28]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [29]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [30]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids=[]
        self.target_ids=[]
        token_ids = tokenizer.encode(txt)
        
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk= token_ids[i:i + max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
        
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,idx):
        return self.input_ids[idx], self.target_ids[idx]

In [31]:
def create_dataloader_v1(txt, 
                         batch_size=4,
                         max_length=256,
                         stride=128,
                         shuffle=True,
                         drop_last=True,
                         num_workers=0):
    
    tokenizer=tiktoken.get_encoding('gpt2')
    dataset=GPTDatasetV1(txt, tokenizer,max_length, stride)
    data_loader=DataLoader(dataset,
                           batch_size=batch_size,
                           shuffle=shuffle,
                           drop_last=drop_last,
                           num_workers=num_workers
                           )
    return data_loader

In [33]:
with open('../data/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [34]:
x, y = first_batch
print(tokenizer.decode(x[0].tolist()), tokenizer.decode(y[0].tolist()))

I HAD always  HAD always thought


In [35]:
x.size()

torch.Size([1, 4])

In [36]:
second_batch = next(data_iter)
print(second_batch)
x, y = second_batch
print(tokenizer.decode(x[0].tolist()), tokenizer.decode(y[0].tolist()))

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]
 HAD always thought AD always thought Jack


In [37]:
dataloader2= create_dataloader_v1(raw_text, batch_size=1, max_length=2, stride=2, shuffle=False)
data_iter2 = iter(dataloader2)
first_batch2 = next(data_iter2)
second_batch2 = next(data_iter2)
print(first_batch2)
print(second_batch2)

[tensor([[ 40, 367]]), tensor([[ 367, 2885]])]
[tensor([[2885, 1464]]), tensor([[1464, 1807]])]


In [38]:
dataloader3= create_dataloader_v1(raw_text, batch_size=1, max_length=8, stride=2, shuffle=False)
data_iter3 = iter(dataloader3)
first_batch3 = next(data_iter3)
second_batch3 = next(data_iter3)
print(first_batch3)
print(second_batch3)

[tensor([[  40,  367, 2885, 1464, 1807, 3619,  402,  271]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899]])]
[tensor([[ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138]]), tensor([[ 1464,  1807,  3619,   402,   271, 10899,  2138,   257]])]


In [39]:
dataloader4= create_dataloader_v1(raw_text, batch_size=2, max_length=8, stride=2, shuffle=False)
data_iter4 = iter(dataloader4)
first_batch4 = next(data_iter4)
second_batch4 = next(data_iter4)
print(first_batch4)
print(second_batch4)

[tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271],
        [ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899],
        [ 1464,  1807,  3619,   402,   271, 10899,  2138,   257]])]
[tensor([[ 1807,  3619,   402,   271, 10899,  2138,   257,  7026],
        [  402,   271, 10899,  2138,   257,  7026, 15632,   438]]), tensor([[ 3619,   402,   271, 10899,  2138,   257,  7026, 15632],
        [  271, 10899,  2138,   257,  7026, 15632,   438,  2016]])]


In [40]:
dataloader= create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargetst:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targetst:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [41]:
input_ids = torch.tensor([2,3,5,1])
vocab_size=6
output_dim=3

torch.manual_seed(123)
embeddings=torch.nn.Embedding(vocab_size, output_dim)
print(embeddings.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [42]:
print(embeddings(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [43]:
print(embeddings(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [44]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

max_length = 4
dataloader = create_dataloader_v1(
    raw_text, 
    batch_size=8,
    max_length = max_length,
    stride=max_length,
    shuffle=False
)
data_iter = iter(dataloader)
inputs, targets=next(data_iter)
print("Token IDs: \n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [45]:
token_embeddings= token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [46]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [47]:
input_embeddings= token_embeddings+ pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


In [48]:
pos_embeddings

tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)

In [49]:
token_embeddings[0]

tensor([[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
        [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
        [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
        [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],
       grad_fn=<SelectBackward0>)

In [50]:
input_embeddings[0]

tensor([[ 2.2288,  0.5619,  0.8286,  ..., -0.6272, -0.2987,  0.8900],
        [ 2.0903, -0.4664, -0.0593,  ...,  0.9115, -1.0493, -1.6473],
        [-0.7158, -0.8304,  1.2494,  ...,  2.3952,  1.8773,  0.8051],
        [ 0.2703,  0.4029,  3.0514,  ...,  0.3595, -1.4548,  0.8310]],
       grad_fn=<SelectBackward0>)

In [51]:
input_embeddings[0]-token_embeddings[0]

tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<SubBackward0>)